# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
    - The PEFT config i have chosen is `LORA` as i feel it is a safe bet and a good place to start. It is also very popular and widely trusted and used. 
    - It is also supported my chosen model.
* Model: 
    - The model i have chosen is `distilbert/distilbert-base-uncased` with a `SequenceClassification` head, based on the Popular `google-bert/bert-base-uncased`.
    - Smaller and faster than bert with comparable accuracy.
* Evaluation approach: 
    - The evaluation method i have chosen is `accuracy` which will measure the closeness of a measured value to a standard or known value
* Fine-tuning dataset: 
    The `Dataset` i have chosen is `rotten_tomatoes` a valid data set to use with 2 features text (input) and label (output).


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Imports


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
from datasets import load_dataset
import evaluate
import torch.nn.functional as F
import torch
from peft import LoraConfig, TaskType
from peft import get_peft_model
from peft import AutoPeftModelForSequenceClassification
import numpy as np



### Load dataset

In [4]:
dataset = load_dataset("rotten_tomatoes")

### Inspect dataset

In [5]:
print("TRAIN:", dataset)
print("TEXT", dataset["train"].features[f"text"])
print("LABEL", dataset["train"].features[f"label"])

TRAIN: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
TEXT Value(dtype='string', id=None)
LABEL ClassLabel(names=['neg', 'pos'], id=None)


In [6]:
example = dataset["train"][0]
print("EXAMPLE:",example)
print("TEXT:",example["text"])
print("LABEL:",example["label"])

EXAMPLE: {'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
TEXT: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
LABEL: 1


### Create label2id, id2label and label_count

In [7]:
# Create label2id, id2label and label_count
labels = dataset["train"].features[f"label"].names
id2label={}
label2id={}
for i, name in enumerate(labels):
    id2label[i] = name
    
for i, name in enumerate(labels):
    label2id[name] = i
    
label_count = len(labels);
print(id2label)
print(label2id)
print(label_count)

{0: 'neg', 1: 'pos'}
{'neg': 0, 'pos': 1}
2


### Load Tokenizer and Model

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# if not tokenizer.pad_token:
#     tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=label_count, id2label=id2label, label2id=label2id
)
# model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(model.config)
print(model)
print(tokenizer)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "neg",
    "1": "pos"
  },
  "initializer_range": 0.02,
  "label2id": {
    "neg": 0,
    "pos": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.32.1",
  "vocab_size": 30522
}

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Tr

### Inspect Tokenizer

In [9]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["text"], truncation=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print("IN", example["text"])
print("OUT", tokens)
# word_ids is used to Map output tokens to their respective input word in the raw examples["tokens"]
print("WORD_IDS", tokenized_input.word_ids())
for i, (token, word_id) in enumerate(zip(tokens, tokenized_input.word_ids())):
    print(f"({token},{word_id})", end=", ")


IN the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
OUT ['[CLS]', 'the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'", 's', 'new', '"', 'conan', '"', 'and', 'that', 'he', "'", 's', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarz', '##ene', '##gger', ',', 'jean', '-', 'cl', '##aud', 'van', 'dam', '##me', 'or', 'steven', 'sega', '##l', '.', '[SEP]']
WORD_IDS [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 29, 30, 31, 32, 33, 33, 34, 35, 35, 36, 37, 38, 38, 39, None]
([CLS],None), (the,0), (rock,1), (is,2), (destined,3), (to,4), (be,5), (the,6), (21st,7), (century,8), (',9), (s,10), (new,11), (",12), (conan,13), (",14), (and,15), (that,16), (he,17), (',18), (s,19), (going,20), (to,21), (make,22), (a,23), (splash,24), (even,25), (gre

### Tokenize and add labels column (which Model expects)

In [10]:
def tokenize_and_align_labels(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True)
    tokenized["labels"] = examples["label"]
    return  tokenized


tokenized_input = dataset.map(tokenize_and_align_labels, batched=True)

example = tokenized_input["train"][0]
text = example["text"]
inputIds = example["input_ids"]
labels = example["labels"]
print("TEXT:",text)
print("CONVERTED_TOKENS:",tokenizer.convert_ids_to_tokens(inputIds))
print("INPUT_IDS:",inputIds)
print("LABELS:",labels)

Map: 100%|██████████| 1066/1066 [00:00<00:00, 8956.82 examples/s]

TEXT: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
CONVERTED_TOKENS: ['[CLS]', 'the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'", 's', 'new', '"', 'conan', '"', 'and', 'that', 'he', "'", 's', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarz', '##ene', '##gger', ',', 'jean', '-', 'cl', '##aud', 'van', 'dam', '##me', 'or', 'steven', 'sega', '##l', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

### Prepare Evalutation Logic


In [11]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### Evaluate model before training;
Evaluate a test example before training the model.
Grab an example with a low accuracy to test after training the model

Build trainer and then evaluate method

In [12]:
def test_custom_examples(model):
    positive_one_review = {"label":"pos","text":"I was deeply moved by this film. The storytelling was exceptional, and the characters felt like old friends. The chemistry between the lead actors was palpable, and the cinematography captured the beauty of the setting. The emotional depth and genuine moments left me smiling long after the credits rolled. A must-watch!"}
    positive_two_review = {"label":"pos","text":"A timeless masterpiece! The acting, direction, and storytelling are exceptional. The characters’ depth and the emotional journey make this film unforgettable. Definitely a must-watch."}
    positive_three_review = {"label":"pos","text":"A delightful musical with stunning visuals and captivating performances. The chemistry between the leads is electric, and the soundtrack is enchanting. A feel-good movie that leaves you humming its tunes."}

    
    negative_one_review = {"label":"neg","text":"I had high hopes for this movie, but it fell flat. The plot was clichéd, and the dialogue felt forced. The pacing was sluggish, and the acting lacked authenticity. I found myself checking my watch, waiting for something interesting to happen. Unfortunately, it never did. Save your time and skip this one."}
    negative_two_review = {"label":"neg","text":"A mind-numbing mess! The plot is convoluted, the dialogue cringe-worthy, and the action sequences lack coherence. It’s a prime example of style over substance."}
    negative_three_review = {"label":"neg","text":"An animated disaster! Shallow characters, a predictable plot, and forced product placements. It’s a soulless cash grab that fails to entertain or engage."}
    
    reviews = [positive_one_review,positive_two_review,positive_three_review,negative_one_review,negative_two_review,negative_three_review]
    
    input = tokenizer([review["text"] for review in reviews], truncation=True, padding=True, return_tensors="pt")

    input.to(device)
    model.to(device)
    output = model(**input)
    print(output.logits.shape)
    predictions = torch.argmax(
        output.logits, dim=1
    )  
    print(predictions.shape)
    predicted_labels = [
        model.config.id2label[prediction.item()] for prediction in predictions
    ]
    for i, (review, label) in enumerate(zip(reviews, predicted_labels)):
        print(f"Review {i}: MATCHED:{label == review['label']} TEXT:{review['text']}")

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="rotten_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_input["train"],
    eval_dataset=tokenized_input["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [14]:
print(trainer.evaluate())
test_custom_examples(model)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 67/67 [00:04<00:00, 14.02it/s]

{'eval_loss': 0.6962642073631287, 'eval_accuracy': 0.4924953095684803, 'eval_runtime': 4.9393, 'eval_samples_per_second': 215.822, 'eval_steps_per_second': 13.565}
torch.Size([6, 2])
torch.Size([6])
Review 0: MATCHED:False TEXT:I was deeply moved by this film. The storytelling was exceptional, and the characters felt like old friends. The chemistry between the lead actors was palpable, and the cinematography captured the beauty of the setting. The emotional depth and genuine moments left me smiling long after the credits rolled. A must-watch!
Review 1: MATCHED:False TEXT:A timeless masterpiece! The acting, direction, and storytelling are exceptional. The characters’ depth and the emotional journey make this film unforgettable. Definitely a must-watch.
Review 2: MATCHED:False TEXT:A delightful musical with stunning visuals and captivating performances. The chemistry between the leads is electric, and the soundtrack is enchanting. A feel-good movie that leaves you humming its tunes.
Revi

You can see above the model has an accuracy of 0.49 (2dp) which is 49% almost random but looking at my custom samples it shows the model is always outputting a negative score, which matches the dataset ratio between negative and positive reviews.

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [15]:
#["lin1","lin2","classifier"]
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, target_modules=['q_lin', 'k_lin', 'v_lin'], inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters();

trainable params: 813,314 || all params: 67,768,324 || trainable%: 1.2001388731407907


### Train

In [16]:
trainer.train()

 47%|████▋     | 501/1068 [01:35<01:46,  5.33it/s]

{'loss': 0.5477, 'learning_rate': 1.0636704119850187e-05, 'epoch': 0.94}


                                                  
 50%|█████     | 534/1068 [01:46<01:42,  5.20it/s]

{'eval_loss': 0.43598470091819763, 'eval_accuracy': 0.8020637898686679, 'eval_runtime': 5.1584, 'eval_samples_per_second': 206.652, 'eval_steps_per_second': 12.988, 'epoch': 1.0}


 94%|█████████▎| 1000/1068 [03:16<00:13,  5.07it/s]

...

                                                   
100%|██████████| 1068/1068 [03:35<00:00,  5.28it/s]

{'eval_loss': 0.42547306418418884, 'eval_accuracy': 0.799249530956848, 'eval_runtime': 5.1559, 'eval_samples_per_second': 206.752, 'eval_steps_per_second': 12.995, 'epoch': 2.0}


100%|██████████| 1068/1068 [03:35<00:00,  4.95it/s]

{'train_runtime': 215.7604, 'train_samples_per_second': 79.069, 'train_steps_per_second': 4.95, 'train_loss': 0.48288458742005996, 'epoch': 2.0}


TrainOutput(global_step=1068, training_loss=0.48288458742005996, metrics={'train_runtime': 215.7604, 'train_samples_per_second': 79.069, 'train_steps_per_second': 4.95, 'train_loss': 0.48288458742005996, 'epoch': 2.0})

# Save model

In [17]:
model.save_pretrained ("rotten_model")

In [18]:
# model.config.to_json_file("rotten_model/adapter_config.json")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

Here i use the index for the test example with the lowest accuracy found before training. 

In [19]:
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("rotten_model", num_labels=label_count, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_input["train"],
    eval_dataset=tokenized_input["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [21]:

print(trainer.evaluate())
test_custom_examples(lora_model)

100%|██████████| 67/67 [00:05<00:00, 12.89it/s]

{'eval_loss': 0.42547306418418884, 'eval_accuracy': 0.799249530956848, 'eval_runtime': 5.285, 'eval_samples_per_second': 201.704, 'eval_steps_per_second': 12.677}
torch.Size([6, 2])
torch.Size([6])
Review 0: MATCHED:True TEXT:I was deeply moved by this film. The storytelling was exceptional, and the characters felt like old friends. The chemistry between the lead actors was palpable, and the cinematography captured the beauty of the setting. The emotional depth and genuine moments left me smiling long after the credits rolled. A must-watch!
Review 1: MATCHED:True TEXT:A timeless masterpiece! The acting, direction, and storytelling are exceptional. The characters’ depth and the emotional journey make this film unforgettable. Definitely a must-watch.
Review 2: MATCHED:True TEXT:A delightful musical with stunning visuals and captivating performances. The chemistry between the leads is electric, and the soundtrack is enchanting. A feel-good movie that leaves you humming its tunes.
Review 3

Now the model has an accuracy of 0.80 (2dp) 80% and is correctly predicting the labels for my custom samples. This improvement for a model only over 2 epochs shows how good the PEFT technique is. It has only 1.2% of the original weights trained upon, which is incredible.